In [3]:
# hydrological packages
import hydroeval as he
from hydrotools.nwm_client import utils # I had to pip install this

# basic packages
import numpy as np
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
import bz2file as bz2

# system packages
from progressbar import ProgressBar
from datetime import datetime, date
import datetime
import pickle as pkl
import warnings
warnings.filterwarnings("ignore")
import platform
import time

# data analysi packages
from scipy import optimize
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import joblib

# deep learning packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#Shared/Utility scripts
import sys
import boto3
import s3fs
sys.path.insert(0, '..') #sys allows for the .ipynb file to connect to the shared folder files

#load access key
HOME = os.path.expanduser('~')
KEYPATH = "NWM_ML/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'streamflow-app-data'
BUCKET = S3.Bucket(BUCKET_NAME)

#s3fs
fs = s3fs.S3FileSystem(anon=False, key=ACCESS['Access key ID'][0], secret=ACCESS['Secret access key'][0])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [4]:
#Get streamstats data 
datapath = f"{HOME}/NWM_ML/Data/input"
file = "Streamstats.csv"
filepath = f"{datapath}/{file}"
try:
    StreamStats = pd.read_csv(filepath)
except:
    print("Data not found, retreiving from AWS S3")
    if not os.path.exists(datapath):
        os.makedirs(datapath, exist_ok=True)
    key = 'Streamstats/Streamstats.csv'      
    S3.meta.client.download_file(BUCKET_NAME, key,filepath)
    StreamStats = pd.read_csv(filepath)

#Get processed training data 
datapath = f"{HOME}/NWM_ML/Data/Processed"
file = "raw_training_data.parquet"
filepath = f"{datapath}/{file}"
try:
    raw_training_data = pd.read_parquet(filepath)
except:
    print("Data not found, retreiving from AWS S3")
    if not os.path.exists(datapath):
        os.makedirs(datapath, exist_ok=True)
    key = "NWM_ML"+datapath.split("NWM_ML",1)[1]+'/'+file       
    S3.meta.client.download_file(BUCKET_NAME, key,filepath)
    raw_training_data = pd.read_parquet(filepath)

raw_training_data.pop('Unnamed: 0')
raw_training_data['station_id'] = raw_training_data['station_id'].astype('str')
raw_training_data.head()

,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY
0,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-28,78.55521,-0.891007,-0.453991,0.0,1.2,55.0,301
1,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-29,98.61146,-0.891007,-0.453991,0.0,1.2,55.0,302
2,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-30,97.60208,-0.891007,-0.453991,0.0,1.1,54.0,303
3,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-31,99.33125,-0.891007,-0.453991,0.0,1.2,54.0,304
4,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-01,95.76354,-0.998630,0.052336,0.0,1.2,54.0,305


### Dataprocessing
* Editing the features based on the feature importance
* Remove headwater stations from dataset
* make sure dates are in datetime format

In [5]:
# Editing the features based on the feature importance should be done here!!!!!!!!!!!!!!!
Training_DF = raw_training_data.copy()
Training_DF.drop(['Mean_Ann_Precip_in', 'Perc_Herbace', 'Perc_Forest',
                        'Mean_Basin_Elev_ft'], axis=1, inplace=True)

#remove headwater stations
headwater_stations = ['10011500', '10109000', '10113500', '10128500', '10131000', '10146400', '10150500', '10154200',
'10172700', '10172800', '10172952']
Training_DF = Training_DF[~raw_training_data['station_id'].isin(headwater_stations)]

#convert dates to datetime format
Training_DF.datetime = pd.to_datetime(Training_DF.datetime)

#Select training data - testing is going to be done on 2020
x_train_temp = Training_DF[Training_DF.datetime.dt.year != 2020]
x_train_temp.pop('station_id')
x_train_temp.pop('datetime')
y_train_temp = x_train_temp['flow_cfs']
x_train_temp.pop('flow_cfs')

#Convert dataframe to numpy, scale, save scalers
y_train = y_train_temp.to_numpy()
x_train = x_train_temp.to_numpy()

modelname = "MLP"
scalername_x = "Area_Perc_Seas_stor_swe_NWM_DOY_scaler_x.save"
scalername_y = "Area_Perc_Seas_stor_swe_NWM_DOY_scaler_y.save"
modelpath = f"{HOME}/NWM_ML/Model/{modelname}"
if not os.path.exists(modelpath):
    os.makedirs(modelpath, exist_ok=True)

scalerfilepath_x = f"{modelpath}/{scalername_x}"
scalerfilepath_y = f"{modelpath}/{scalername_y}"

scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
joblib.dump(scaler, scalerfilepath_x)

scaler = MinMaxScaler()
y_scaled_train = scaler.fit_transform(y_train.reshape(-1, 1))
joblib.dump(scaler, scalerfilepath_y)  
y_scaled_train.shape

(128879, 1)

### Set up Testing year
* Select year(s) not used in training
* Convert to numpy array
* Load scaler and scale data

In [6]:
#Get water year for testing from larger dataset
x_test_temp = Training_DF[Training_DF.datetime.dt.year == 2020]
x_test_temp_1 = x_test_temp.copy()
station_index_list = x_test_temp_1['station_id']
x_test_temp_1.pop('station_id')
x_test_temp_1.pop('datetime')

#Get target variable (y) and convert to numpy arrays
y_test_temp_1 = x_test_temp_1['flow_cfs']
x_test_temp_1.pop('flow_cfs')
x_test_1_np = x_test_temp_1.reset_index(drop=True).to_numpy()
y_test_1_np = y_test_temp_1.reset_index(drop=True).to_numpy()

#load scalers and scale
modelname = "MLP"
scalername_x = "Area_Perc_Seas_stor_swe_NWM_DOY_scaler_x.save"
scalername_y = "Area_Perc_Seas_stor_swe_NWM_DOY_scaler_y.save"
modelpath = f"{HOME}/NWM_ML/Model/{modelname}"
scalerfilepath_x = f"{modelpath}/{scalername_x}"
scalerfilepath_y = f"{modelpath}/{scalername_y}"

#load scalers
scaler_x = joblib.load(scalerfilepath_x)
scaler_y = joblib.load(scalerfilepath_y)

#scale the testing data
x_test_1_scaled = scaler_x.fit_transform(x_test_1_np)
y_scaled_test_1 = scaler_y.fit_transform(y_test_1_np.reshape(-1, 1))


### Set up model training framework

In [7]:
# %% MLP

n_targets = 1
tries = 10
#model performance metrics
cri_temp_nse = np.zeros([3, n_targets, tries])
cri_temp_rmse = np.zeros([3, n_targets, tries])
cri_temp_r2 = np.zeros([3, n_targets, tries])
cri_temp_kge = np.zeros([3, n_targets, tries])
cri_temp_lognse = np.zeros([3, n_targets, tries])

# Convert to tensor for PyTorch
x_train_scaled_t = torch.Tensor(x_train_scaled)
y_train_scaled_t = torch.Tensor(y_scaled_train)
#Make sure the tensors on are the respective device (cpu/gpu)
x_train_scaled_t = x_train_scaled_t.to(device)
y_train_scaled_t = y_train_scaled_t.to(device)


## Train the model

In [6]:
#Train the model
model_path = f"{HOME}/NWM_ML/Model/MLP"

start_time = time.time()

# Hyperparameters
epochs = 20
batch_size = 100
learning_rate = 0.001
decay = 1e-2
validation_split = 0.2
neurons = 150
LD1=128
LD2=128
LD3=64
LD4=64
LD5=32
LD6=16
LD7=5

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(x_train_scaled_t, y_train_scaled_t)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False )

# Build the model
model = nn.Sequential(
    nn.Linear(12, LD1),
    nn.ReLU(),
    nn.Linear(LD1, LD2),
    nn.ReLU(),
    nn.Linear(LD2, LD3),
    nn.ReLU(),
    nn.Linear(LD3, LD4),
    nn.ReLU(),
    nn.Linear(LD4, LD5),
    nn.ReLU(),
    nn.Linear(LD5, LD6),
    nn.ReLU(),
    nn.Linear(LD6, 1)
).to(device)


# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop

for epoch in range(epochs):
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

print('finish')
print("Run Time:" + " %s seconds " % (time.time() - start_time))

#save model
if os.path.exists(model_path) == False:
    os.mkdir(model_path)
torch.save(model.state_dict(), f"{model_path}/mlp_model.pkl")

Epoch 1/20, Loss: 0.0011841386958609057
Epoch 2/20, Loss: 0.0009998900827772198
Epoch 3/20, Loss: 0.0010072439865052032
Epoch 4/20, Loss: 0.0008762492653219075
Epoch 5/20, Loss: 0.0007190352959022427
Epoch 6/20, Loss: 0.0007379480756709126
Epoch 7/20, Loss: 0.0007507959719313107
Epoch 8/20, Loss: 0.0005918934312767292
Epoch 9/20, Loss: 0.00048261876871947564
Epoch 10/20, Loss: 0.00055188945301148
Epoch 11/20, Loss: 0.0005203173422703178
Epoch 12/20, Loss: 0.0005119359537504274
Epoch 13/20, Loss: 0.00047021937113725425
Epoch 14/20, Loss: 0.0005001171873563373
Epoch 15/20, Loss: 0.0004531849028610537
Epoch 16/20, Loss: 0.0004646811413347393
Epoch 17/20, Loss: 0.0004428204812767694
Epoch 18/20, Loss: 0.00046341442878239473
Epoch 19/20, Loss: 0.00041467104534421257
Epoch 20/20, Loss: 0.00042162290621680784
finish
Run Time: 56.165053606033325 seconds 


## Load the model for evaluation

In [8]:
# Build and load the model
model_path = f"{HOME}/NWM_ML/Model/MLP"
# Hyperparameters
epochs = 20
batch_size = 100
learning_rate = 0.001
decay = 1e-2
validation_split = 0.2
neurons = 150
LD1=128
LD2=128
LD3=64
LD4=64
LD5=32
LD6=16
LD7=5

#device = torch.device('cpu') # for some reason had to change to cpu
models = nn.Sequential(
    nn.Linear(12, LD1),
    nn.ReLU(),
    nn.Linear(LD1, LD2),
    nn.ReLU(),
    nn.Linear(LD2, LD3),
    nn.ReLU(),
    nn.Linear(LD3, LD4),
    nn.ReLU(),
    nn.Linear(LD4, LD5),
    nn.ReLU(),
    nn.Linear(LD5, LD6),
    nn.ReLU(),
    nn.Linear(LD6, 1)
).to(device)

models.load_state_dict(torch.load(f"{model_path}/mlp_model.pkl"))

#put the model scores into a dataframe for comparison
mod = 'MLP'
#Evaluation columns for prediction time series
cols = ['USGSid', 'NHDPlusid', 'NWM_rmse', f"{mod}_rmse", 'NWM_pbias', f"{mod}_pbias", 
        'NWM_kge', f"{mod}__kge", 'NWM_mape',  f"{mod}_mape"]

#Evaluation columns for accumulated supply time series
supcols = ['USGSid', 'NHDPlusid', 'NWM_rmse', f"{mod}_rmse", 'NWM_pbias', f"{mod}_pbias", 
        'NWM_kge', f"{mod}__kge", 'NWM_mape',  f"{mod}_mape", 'Obs_vol', 'NWM_vol', f"{mod}_vol",
        'NWM_vol_err', f"{mod}_vol_err", 'NWM_vol_Perc_diff', f"{mod}_vol_Perc_diff"]


EvalDF = pd.DataFrame(columns = cols)
SupplyEvalDF = pd.DataFrame(columns = supcols)

# Make a prediction for each location, save as compressed pkl file, and send predictions to AWS for use in CSES

In [9]:
#get annual supply diffs
cfsday_AFday = 1.983
mod = 'MLP'
year = 2020


Preds_Dict = {}
for station_number in station_index_list.drop_duplicates():
  #print(station_number)
  index = station_index_list == station_number
  X_test = x_test_temp_1[index]
  X_test_scaled_t = torch.Tensor(x_test_1_scaled[index])
  X_test_scaled_t = X_test_scaled_t.to(device)
  l = len(y_test_temp_1.values)
  y_test = torch.Tensor(np.array(y_test_temp_1.values).reshape(l,1))
  y_test = y_test.to(device)

  # Evaluation
  models.eval()
  with torch.no_grad():
      predictions_scaled= models(X_test_scaled_t)

  # Invert scaling for actual
  predictions = scaler_y.inverse_transform(predictions_scaled.to('cpu').numpy())
  predictions[predictions<0] = 0

  #print('Model Predictions complete')

  predictions = pd.DataFrame(predictions, columns=['MLP_flow'])

  #save predictions, need to convert to NHDPlus reach - Need to add Datetime column and flow predictions
  #make daterange
  dates = pd.date_range(pd.to_datetime("2020-01-01"), periods=len(predictions)).strftime("%Y-%m-%d").tolist()
  predictions['Datetime'] = dates
    
  #get reach id for model eval
  nhdreach = utils.crosswalk(usgs_site_codes=station_number)
  nhdreach = nhdreach['nwm_feature_id'].iloc[0]

  #put columns in correct order
  cols = ['Datetime', 'MLP_flow']
  predictions = predictions[cols]

  #save predictions to AWS so we can use CSES
  state = StreamStats['state_id'][StreamStats['NWIS_site_id'].astype(str)== station_number].values[0].lower()
  csv_key = f"{modelname}/NHD_segments_{state}.h5/{modelname[:3]}_{nhdreach}.csv"
  predictions.to_csv(f"s3://{BUCKET_NAME}/{csv_key}", index = False,  storage_options={'key': ACCESS['Access key ID'][0],
                           'secret': ACCESS['Secret access key'][0]})

  #Concat DFS and put into dictionary
  x_test_temp['nwm_feature_id'] = nhdreach
  Dfs = [predictions.reset_index(drop=True),x_test_temp[x_test_temp['station_id']==station_number].reset_index(drop=True)]
  Preds_Dict[station_number] = pd.concat(Dfs, axis=1)

  #reorganize columns
  Preds_Dict[station_number].pop('datetime')
  Preds_Dict[station_number].insert(1, "MLP_flow", Preds_Dict[station_number].pop("MLP_flow"))
  Preds_Dict[station_number].insert(1, "NWM_flow", Preds_Dict[station_number].pop("NWM_flow"))
  Preds_Dict[station_number].insert(1, "flow_cfs", Preds_Dict[station_number].pop("flow_cfs"))
  Preds_Dict[station_number].insert(1, "nwm_feature_id", Preds_Dict[station_number].pop("nwm_feature_id"))
  Preds_Dict[station_number].insert(1, "station_id", Preds_Dict[station_number].pop("station_id"))

  #push data to AWS so we can use CSES
  
  
#save predictions as compressed pkl file
pred_path = f"{HOME}/NWM_ML/Predictions/Hindcast/{mod}/{year}"
file_path = f"{pred_path}/{mod}_predictions.pkl"
if os.path.exists(pred_path) == False:
    os.mkdir(pred_path)

# with bz2.BZ2File(file_path, 'w') as f:
#   pkl.dump(Preds_Dict, f)


with open(file_path, 'wb') as handle:
    pkl.dump(Preds_Dict, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [15]:
Preds_Dict['10157500']

,Datetime,station_id,nwm_feature_id,flow_cfs,NWM_flow,MLP_flow,Lat,Long,Drainage_area_mi2,Perc_Develop,Perc_Imperv,Perc_Slop_30,s1,s2,storage,swe,DOY
0,2020-01-01,10157500,10375690,0.225000,49.0,29.372356,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.438371,0.898794,0.0,0.0,1
1,2020-01-02,10157500,10375690,0.270000,49.0,29.352655,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.438371,0.898794,0.0,0.0,2
2,2020-01-03,10157500,10375690,0.275000,48.0,29.287912,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.438371,0.898794,0.0,0.0,3
3,2020-01-04,10157500,10375690,0.250000,48.0,29.268217,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.438371,0.898794,0.0,0.0,4
4,2020-01-05,10157500,10375690,0.285000,48.0,29.248516,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.438371,0.898794,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2020-09-19,10157500,10375690,0.010000,63.0,33.948734,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.529919,-0.848048,0.0,0.0,270
263,2020-09-20,10157500,10375690,0.010000,62.0,33.817249,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.529919,-0.848048,0.0,0.0,271
264,2020-09-21,10157500,10375690,0.010833,62.0,33.740856,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.529919,-0.848048,0.0,0.0,272
265,2020-09-22,10157500,10375690,0.017273,62.0,33.664471,40.460789,-111.472687,49.8,2.37,0.16,49.1,-0.529919,-0.848048,0.0,0.0,273


In [ ]:
#       # merge
#     Eval_DF_mine = pd.concat(Dfs, axis=1)
#     prediction_columns = [ 'NWM_flow', 'MLP_flow']

#     #plot the predictions
#    # Model_Evaluation_Plots(Eval_DF_mine, prediction_columns)

#     #Get RMSE from the model
#     rmse = RMSE(Eval_DF_mine, prediction_columns)

#     #Get Mean Absolute Percentage Error from the model
#     mape = MAPE(Eval_DF_mine, prediction_columns)

#     #Get Percent Bias from the model
#     pbias = PBias(Eval_DF_mine, prediction_columns)

#     #Get Kling-Gutz Efficiency from the model
#     kge = KGE(Eval_DF_mine, prediction_columns)
    
#       #Get Volumetric values
#     Eval_DF_mine['Datetime'] = pd.to_datetime(dates)
#     Eval_DF_mine.set_index('Datetime', inplace = True, drop =True)
#     flowcols = [f"{mod}_flow", 'flow_cfs', 'NWM_flow']
#     SupplyEval = Eval_DF_mine[flowcols].copy()
#     SupplyEval = SupplyEval*cfsday_AFday
#     #set up cumulative monthly values
#     SupplyEval['Year'] = SupplyEval.index.year

#     for site in flowcols:
#         SupplyEval[site] = SupplyEval.groupby(['Year'])[site].cumsum()  

#     EOY_mod_vol_af = SupplyEval[f"{mod}_flow"].iloc[-1]
#     EOY_obs_vol_af = SupplyEval["flow_cfs"].iloc[-1]
#     EOY_nwm_vol_af = SupplyEval[f"NWM_flow"].iloc[-1]
#     NWM_vol_diff_af = EOY_nwm_vol_af - EOY_obs_vol_af
#     Mod_vol_diff_af = EOY_mod_vol_af - EOY_obs_vol_af
#     NWM_Perc_diff = (NWM_vol_diff_af/EOY_obs_vol_af)*100
#     Mod_Perc_diff = (Mod_vol_diff_af/EOY_obs_vol_af)*100
    
#      #Get Performance Metrics from the model
#     Srmse = RMSE(SupplyEval, prediction_columns)
#     Smape = MAPE(SupplyEval, prediction_columns)
#     Spbias = PBias(SupplyEval, prediction_columns)
#     Skge = KGE(SupplyEval, prediction_columns)
    
    
#     #save model performance
#     sitestats = [station_number, nhdreach, rmse[0], rmse[1],  pbias[0], pbias[1], kge[0], kge[1], mape[0],mape[1]]
#     EvalDF.loc[len(EvalDF)] = sitestats
    
#     Supplystats = [station_number, nhdreach, Srmse[0], Srmse[1],  Spbias[0], Spbias[1], Skge[0], Skge[1], Smape[0],  
#                  Smape[1],EOY_obs_vol_af, EOY_nwm_vol_af,EOY_mod_vol_af,NWM_vol_diff_af,Mod_vol_diff_af, NWM_Perc_diff, Mod_Perc_diff ]
#     SupplyEvalDF.loc[len(SupplyEvalDF)] = Supplystats
    
    
    
    
#     #put prediction DF into dictionary
#     Eval_DF_mine.sort_values(by=['DOY'], inplace=True)
#     Eval_DF_mine['Datetime'] = pd.to_datetime(dates)
#     Eval_DF_mine.set_index('Datetime', inplace = True)
#     SitesDict[nhdreach] = Eval_DF_mine
    
# #save model results
# EvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Performance.csv")   
# SupplyEvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Supply_Performance.csv")


# print("Model Performance for Daily cfs")
# display(EvalDF)   
# print("Model Performance for Daily Accumulated Supply (Acre-Feet)")
# display(SupplyEvalDF )

In [ ]:
#save model results
EvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Performance.csv")

for site in SitesDict.keys():
    SitesDict[site].rename(columns = {'flow_cfs':'Obs_flow'}, inplace =  True)



In [ ]:
import pickle as pkl
save_path = f"{HOME}/NWM-ML/Predictions/Hindcast/{mod}"
with open(f'{save_path}/Pred_dict.pkl', 'wb') as handle:
    pkl.dump(SitesDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
mod = 'MLP'
save_path = f"{HOME}/NWM-ML/Predictions/Hindcast/{mod}"
with open(f'{save_path}/Pred_dict.pkl', 'rb') as handle:
    SitesDict = pkl.load(handle)

In [ ]:
import importlib
importlib.reload(FigureGenerator)

In [ ]:
import FigureGenerator

model = 'MLP'
plotname = 'MLP_TS_plot'
freq = 'D'
supply = True
fill_between = True
title = 'Observed and Modeled flows for NHDPlus Reaches \n with Upstream Reservoirs in the Great Salt Lake Basin'
FigureGenerator.TS_plot(SitesDict, model, plotname, title, freq, supply, fill_between)

In [ ]:
plotname = 'MLP_ParityPlot'
FigureGenerator.Parity_plot(SitesDict, model, plotname)

In [ ]:
import AWS_transfer
model = 'MLP'
AWS_transfer.Predictions2AWS(model)